<a href="https://colab.research.google.com/github/SourCherries/VAE-GAN-CelebA/blob/master/learn_tf_conv2d_valid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
import numpy as np
import tensorflow as tf

After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow 1.x selected.


In [2]:
# Rufin's VAE-GAN-Celeb uses padding = 'SAME'
#   Should try with that.
#   Current results work out will with 'VALID'

def get_conv_tf(x, w, bias, S=1, padding='VALID'):
  print("Preparing input and kernel for convolution in TF *****************************************************\n")
  S = 1; print("S (stride) =\t" + str(S))
  print("w:\t" + str(w.shape))
  x=np.array(x).astype(np.float32); print("x:\t" + str(x.shape))
  x_tf=tf.constant(np.expand_dims(x,axis=0),dtype=tf.float32); print("x_tf after expand_dims:\t" + str(x_tf.shape))
  f_tf=tf.constant(np.expand_dims(w,axis=0),dtype=tf.float32); print("f_tf:after expand_dims:\t" + str(f_tf.shape))
  x_tf=tf.reshape(x_tf,shape=(1,x.shape[0],x.shape[1],x.shape[2])); print("x_tf after reshape:\t" + str(x_tf.shape))
  f_tf=tf.reshape(f_tf,shape=(w.shape[0],w.shape[1],w.shape[2],1)); print("f_tf after reshape:\t" + str(f_tf.shape))
  b_tf=tf.constant(bias,dtype=tf.float32); print("b_tf:\t" + str(b_tf.shape))
  conv_tf=tf.nn.conv2d(x_tf,f_tf,strides=[1,S,S,1],padding='VALID')
  conv_tf=tf.nn.bias_add(conv_tf,b_tf)
  return (x_tf, f_tf, b_tf, conv_tf)
# Modified from:
# # https://towardsdatascience.com/understanding-convolutions-by-hand-vs-tensorflow-8e64053f673e

In [3]:
def string_of_3d_matrix(m):
  if len(m.shape) == 2:
    m = np.expand_dims(m, axis=2)
  m_string = ''
  for i in range(m.shape[2]):
    m_string += str(m[:,:,i]) + "\n"
  return m_string

In [4]:
def show_results(x, x_tf, w, f_tf, conv_tf):
  print("Examine intermediate stages of results *****************************************************\n")

  x_string = string_of_3d_matrix(x)
  print("x = ")
  print(x_string)

  x_tf_string = string_of_3d_matrix(tf.Session().run( x_tf )[0,:,:,:])
  print("x_tf = ")
  print(x_tf_string)

  w_string = string_of_3d_matrix(w)
  print("w = ")
  print(w_string)

  CHANNEL_OUT = 0

  fout = tf.Session().run( f_tf )
  f_tf_string = string_of_3d_matrix(fout[:,:,:,CHANNEL_OUT].squeeze())
  print(f"f_tf[:,:,:,{CHANNEL_OUT}] =")
  print(f_tf_string)

  cout = tf.Session().run( conv_tf )
  c_string = str(cout[:, :, :, CHANNEL_OUT].squeeze())
  print(f"cout[:,:,:,{CHANNEL_OUT}] =\n{c_string}\n\n")
  return (fout, cout)
# For conversion of TF tensor object to numpy array see:
# https://stackoverflow.com/questions/34097281/convert-a-tensor-to-numpy-array-in-tensorflow

# Padding type might be crucial

In [14]:
# padding_type = 'VALID'  # results work out
padding_type = 'SAME'   # what RVR uses - works same?

In [15]:
# Simple case with single-channel image
x = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
x = np.expand_dims(x,axis=2)

w = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 1]])
w = np.expand_dims(w,axis=2)

bias = np.array([100])

x_tf, f_tf, b_tf, conv_tf = get_conv_tf(x, w, bias, S=1, padding=padding_type)

fout, cout = show_results(x, x_tf, w, f_tf, conv_tf)

print("Confirmation *********************************************************")
if cout[:,:,:,0].squeeze() == (9 + bias.squeeze()):
  print("Worked as expected")

Preparing input and kernel for convolution in TF *****************************************************

S (stride) =	1
w:	(3, 3, 1)
x:	(3, 3, 1)
x_tf after expand_dims:	(1, 3, 3, 1)
f_tf:after expand_dims:	(1, 3, 3, 1)
x_tf after reshape:	(1, 3, 3, 1)
f_tf after reshape:	(3, 3, 1, 1)
b_tf:	(1,)
Examine intermediate stages of results *****************************************************

x = 
[[1 2 3]
 [4 5 6]
 [7 8 9]]

x_tf = 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]

w = 
[[0 0 0]
 [0 0 0]
 [0 0 1]]

f_tf[:,:,:,0] =
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]

cout[:,:,:,0] =
109.0


Confirmation *********************************************************
Worked as expected


In [16]:
# Two channel image
v = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
v = np.expand_dims(v, axis=2)
x = np.concatenate([v, v + 10], axis=2)

blank = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
blank = np.expand_dims(blank, axis=2)
v = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 1]])
v = np.expand_dims(v, axis=2)
w = np.concatenate([blank, v], axis=2)

bias = np.array([100])

x_tf, f_tf, b_tf, conv_tf = get_conv_tf(x, w, bias, S=1, padding=padding_type)

fout, cout = show_results(x, x_tf, w, f_tf, conv_tf)

print("Confirmation *********************************************************")
# 9 for position, 10 for second channel
if cout[:,:,:,0].squeeze() == (9 + 10 + bias.squeeze()):
  print("Worked as expected")

Preparing input and kernel for convolution in TF *****************************************************

S (stride) =	1
w:	(3, 3, 2)
x:	(3, 3, 2)
x_tf after expand_dims:	(1, 3, 3, 2)
f_tf:after expand_dims:	(1, 3, 3, 2)
x_tf after reshape:	(1, 3, 3, 2)
f_tf after reshape:	(3, 3, 2, 1)
b_tf:	(1,)
Examine intermediate stages of results *****************************************************

x = 
[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[11 12 13]
 [14 15 16]
 [17 18 19]]

x_tf = 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
[[11. 12. 13.]
 [14. 15. 16.]
 [17. 18. 19.]]

w = 
[[0 0 0]
 [0 0 0]
 [0 0 0]]
[[0 0 0]
 [0 0 0]
 [0 0 1]]

f_tf[:,:,:,0] =
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]

cout[:,:,:,0] =
119.0


Confirmation *********************************************************
Worked as expected


In [17]:
# Single channel image with larger size 
x = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
x = np.pad(x, ((1, 1), (1, 1)))
x = np.expand_dims(x,axis=2)

w = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 1]])
w = np.expand_dims(w,axis=2)

bias = np.array([100])

x_tf, f_tf, b_tf, conv_tf = get_conv_tf(x, w, bias, S=1, padding=padding_type)

fout, cout = show_results(x, x_tf, w, f_tf, conv_tf)

print("Confirmation *********************************************************")
answer = np.array([[105, 106, 100], [108, 109, 100], [100, 100, 100]])
if (cout[:,:,:,0].squeeze() == (answer)).all():
  print("Worked as expected")

Preparing input and kernel for convolution in TF *****************************************************

S (stride) =	1
w:	(3, 3, 1)
x:	(5, 5, 1)
x_tf after expand_dims:	(1, 5, 5, 1)
f_tf:after expand_dims:	(1, 3, 3, 1)
x_tf after reshape:	(1, 5, 5, 1)
f_tf after reshape:	(3, 3, 1, 1)
b_tf:	(1,)
Examine intermediate stages of results *****************************************************

x = 
[[0 0 0 0 0]
 [0 1 2 3 0]
 [0 4 5 6 0]
 [0 7 8 9 0]
 [0 0 0 0 0]]

x_tf = 
[[0. 0. 0. 0. 0.]
 [0. 1. 2. 3. 0.]
 [0. 4. 5. 6. 0.]
 [0. 7. 8. 9. 0.]
 [0. 0. 0. 0. 0.]]

w = 
[[0 0 0]
 [0 0 0]
 [0 0 1]]

f_tf[:,:,:,0] =
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]

cout[:,:,:,0] =
[[105. 106. 100.]
 [108. 109. 100.]
 [100. 100. 100.]]


Confirmation *********************************************************
Worked as expected
